In [1]:
import nibabel as nib
import numpy as np
import glob
from tqdm import tqdm

In [2]:
nii_files = glob.glob("../../Data/Temp/ADNI/**/*.nii", recursive=True)
images = []
for nii_file in nii_files:
    try:
        img_ = nib.load(nii_file)
        images.append(img_)
    except: pass

print(f"{len(images)} scans with shape {images[0].shape}")


59 scans with shape (256, 256, 170)


In [3]:
def check_and_correct_rotation(img):
    bottom_edge_brightness = np.mean(img[-1, :, :])
    right_edge_brightness = np.mean(img[:, -1, :])
    # Bottom edge SHOULD have more brightness
    if right_edge_brightness > bottom_edge_brightness:
        img = np.transpose(img, axes=(1, 0, 2))  # Swap x and y axes
        img = img[:, :, ::-1]  # Reverse the swapped axis
    return img

In [10]:

# Function to add padding to match the target shape (256, 256, 170)
def add_padding(image, target_shape=(256, 256, 176)):
    image_data = image.get_fdata()
    affine = image.affine
    
    # Determine padding for each dimension
    padding = []
    for dim, size in enumerate(image_data.shape):
        total_padding = target_shape[dim] - size
        if dim == 2:  # z-dimension: Avoid padding where the neck is cut off (inferior side)
            pad_before = total_padding
            pad_after = total_padding - pad_before
            padding.append((pad_before, pad_after - 1 if pad_after > 0 else 0))
        else:  # x and y dimensions: center the padding
            pad_before = total_padding // 2
            pad_after = total_padding - pad_before
            padding.append((pad_before, pad_after))
    
    # Apply padding
    padded_data = np.pad(image_data, padding, mode='constant', constant_values=0)
    
    # Adjust affine for padding
    new_affine = affine.copy()
    new_affine[:3, 3] -= np.array([p[0] for p in padding]) * np.array(image.header.get_zooms())
    
    # Create new NIfTI image with padding
    padded_image = nib.Nifti1Image(padded_data, new_affine)
    return padded_image

# Process all images with rotation correction and padding
resampled_images = []
for img in tqdm(images, desc="Processing Images", unit="image"):
    corrected_img = check_and_correct_rotation(img.get_fdata())
    corrected_nifti = nib.Nifti1Image(corrected_img, img.affine)
    padded_img = add_padding(corrected_nifti)
    resampled_images.append(padded_img)

Processing Images: 100%|██████████| 59/59 [00:35<00:00,  1.67image/s]


In [ ]:
for i, img in enumerate(resampled_images):
    nib.save(img, f'../../Data/Pre-processed/Scaled and padded/resampled_image_{i}.nii')

: 

In [11]:
resampled_images[20].shape

(256, 256, 176)